### 0. Library Import

In [ ]:
## installation
! pip install efficientnet

In [ ]:
#101
import os
import sys
import pandas as pd
import numpy as np
from skimage.io import imread

#plot
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

#files
from keras.preprocessing import image
import zipfile
from sklearn.model_selection import train_test_split

#macine learning
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

#model evaluation
from sklearn.metrics import precision_recall_curve, auc, f1_score,accuracy_score, precision_score, recall_score
from keras.callbacks import Callback

##pre-trained model
#efficientNet
from efficientnet.tfkeras import EfficientNetB0
from efficientnet.tfkeras import center_crop_and_resize, preprocess_input

In [ ]:
tf.__version__

### 1. Data Import

In [ ]:
# check data availability
PATH="../input/iwildcam-2019-fgvc6/"
os.listdir(PATH)

In [ ]:
# class
classes_wild = {0: 'empty', 1: 'deer', 2: 'moose', 3: 'squirrel', 4: 'rodent', 5: 'small_mammal', \
                6: 'elk', 7: 'pronghorn_antelope', 8: 'rabbit', 9: 'bighorn_sheep', 10: 'fox', 11: 'coyote', \
                12: 'black_bear', 13: 'raccoon', 14: 'skunk', 15: 'wolf', 16: 'bobcat', 17: 'cat',\
                18: 'dog', 19: 'opossum', 20: 'bison', 21: 'mountain_goat', 22: 'mountain_lion'}

In [ ]:
path_to_zip = "../input/iwildcam-2019-fgvc6/train_images.zip"
directory_to_extract="../output/kaggle/working/train_images"

with zipfile.ZipFile(path_to_zip, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract)

zip_ref.close()

In [ ]:
train_image_files = list(os.listdir(os.path.join(directory_to_extract)))
print("Number of image files: train:{}".format(len(train_image_files)))

### 2. Data Wrangling

In [ ]:
train_df = pd.read_csv(os.path.join(PATH, 'train.csv'))
test_df = pd.read_csv(os.path.join(PATH, 'test.csv'))

display(train_df.head())
display(test_df.head())

In [ ]:
display(train_df.info())
display(test_df.info())

In [ ]:
fig = plt.figure(figsize=(25, 16))
for i,im_path in enumerate(train_image_files[:16]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open(os.path.join(directory_to_extract,im_path))
    im = im.resize((480,270))
    plt.imshow(im)

### 3. Feature Engineering

In [ ]:
train_df.head()

In [ ]:
train_df['classes_wild'] = train_df['category_id'].apply(lambda cw: classes_wild[cw])

In [ ]:
train_df.head()

In [ ]:
## dataset splitting
x_train, x_test = train_test_split(train_df, test_size=0.2, random_state=42)
x_train.shape, x_test.shape

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

train_datagen=ImageDataGenerator(rescale=1./255, 
                                 validation_split=0.25,
                                 #horizontal_flip = True,    
                                 #zoom_range = 0.3,
                                 #width_shift_range = 0.3,
                                 #height_shift_range=0.3
                                )

In [ ]:
train_generator=train_datagen.flow_from_dataframe(
                    dataframe=x_train,
                    directory="../output/kaggle/working/train_images/",
                    x_col="file_name",
                    y_col="classes_wild",
                    subset="training",
                    batch_size=64,
                    seed=424,
                    shuffle=True,
                    class_mode="categorical",
                    target_size=(128, 128))

valid_generator=train_datagen.flow_from_dataframe(
                    dataframe=x_train,
                    directory="../output/kaggle/working/train_images/",
                    x_col="file_name",
                    y_col="classes_wild",
                    subset="validation",
                    batch_size=64,
                    seed=424,
                    shuffle=True,
                    class_mode="categorical",
                    target_size=(128, 128))

In [ ]:
print(train_generator.class_indices)

In [ ]:
print(valid_generator.class_indices)

### 4. Modeling

In [ ]:
num_classes = train_df['classes_wild'].nunique()

##### 4.1. EfficientNet

In [ ]:
pre_trained_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(128,128,3))

for layer in pre_trained_model.layers:
    layer.trainable = False
    
pre_trained_model.summary()

In [ ]:
# tuning on pre-trained model
effnet_model = tf.keras.models.Sequential()
effnet_model.add(pre_trained_model)
effnet_model.add(tf.keras.layers.GlobalAveragePooling2D())    
effnet_model.add(tf.keras.layers.Dense(num_classes, activation="softmax") )
effnet_model.summary()

opt = tf.keras.optimizers.Adam(lr=0.005, decay=1e-6)
effnet_model.compile(optimizer = opt, 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')


In [ ]:
history = effnet_model.fit(
            train_generator,
            validation_data = valid_generator,
            steps_per_epoch = 100,
            epochs = 20,
            batch_size=64,
            validation_steps = 50,
            callbacks = [early]
)

In [ ]:
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [ ]:
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 211)
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 212)

### Testing Dataset

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
                    dataframe=x_test,
                    directory="../output/kaggle/working/train_images/",
                    x_col="file_name",
                    y_col="classes_wild",
                    batch_size=64,
                    seed=424,
                    shuffle=True,
                    class_mode="categorical",
                    target_size=(128,128))

test_loss, test_acc =effnet_model.evaluate_generator(test_generator, steps=32)
print('test_loss_effnet: {} and test_acc_effnet: {} '.format(test_loss, test_acc))

In [ ]:
# # effnet
converter = tf.lite.TFLiteConverter.from_keras_model(effnet_model)
converter.experimental_new_converter = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
effnet_tflite_model = converter.convert()


model_name = "effnet_tflite_model_b0"
open(f"{model_name}.tflite" , "wb").write(effnet_tflite_model)

### Unfreeze Layers Block7

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(pre_trained_model, to_file='pretrained_model.png', show_shapes=True)
from IPython.display import Image
Image(filename='pretrained_model.png') 

In [ ]:
for layer in pre_trained_model.layers:
    print(layer.name)

In [ ]:
pre_trained_model.trainable = True

set_trainable = False
for layer in pre_trained_model.layers:
    if layer.name == 'block7a_expand_conv':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
effnet_model.compile(optimizer = opt, 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

history = effnet_model.fit(
            train_generator,
            validation_data = valid_generator,
            steps_per_epoch = 100,
            epochs = 20,
            batch_size=64,
            validation_steps = 50,
            callbacks = [early]
)

In [ ]:
effnet_model.summary()

In [ ]:
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 211)
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 212)

In [ ]:
test_loss, test_acc =effnet_model.evaluate_generator(test_generator, steps=32)
print('test_loss_effnet: {} and test_acc_effnet: {} '.format(test_loss, test_acc))

In [ ]:
# # effnet
converter = tf.lite.TFLiteConverter.from_keras_model(effnet_model)
converter.experimental_new_converter = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
effnet_tflite_model = converter.convert()


model_name = "effnet_tflite_model_b0"
open(f"{model_name}.tflite" , "wb").write(effnet_tflite_model)

### Unfreeze: block6c_add

In [ ]:
pre_trained_model.trainable = True

set_trainable = False
for layer in pre_trained_model.layers:
    if layer.name == 'block6c_add':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
effnet_model.compile(optimizer = opt, 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

history = effnet_model.fit(
            train_generator,
            validation_data = valid_generator,
            steps_per_epoch = 100,
            epochs = 20,
            batch_size=64,
            validation_steps = 50,
            callbacks = [early]
)

In [ ]:
effnet_model.summary()

In [ ]:
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 211)
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 212)

In [ ]:
test_loss, test_acc =effnet_model.evaluate_generator(test_generator, steps=32)
print('test_loss_effnet: {} and test_acc_effnet: {} '.format(test_loss, test_acc))

In [ ]:
# # effnet
converter = tf.lite.TFLiteConverter.from_keras_model(effnet_model)
converter.experimental_new_converter = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
effnet_tflite_model = converter.convert()


model_name = "effnet_tflite_model_b2"
open(f"{model_name}.tflite" , "wb").write(effnet_tflite_model)

### Unfreeze: block6a_expand_conv

In [ ]:
pre_trained_model.trainable = True

set_trainable = False
for layer in pre_trained_model.layers:
    if layer.name == 'block6a_expand_conv':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
effnet_model.compile(optimizer = opt, 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

history = effnet_model.fit(
            train_generator,
            validation_data = valid_generator,
            steps_per_epoch = 100,
            epochs = 20,
            batch_size=64,
            validation_steps = 50,
            callbacks = [early]
)

In [ ]:
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 211)
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 212)

In [ ]:
effnet_model.summary()

In [ ]:
test_loss, test_acc =effnet_model.evaluate_generator(test_generator, steps=32)
print('test_loss_effnet: {} and test_acc_effnet: {} '.format(test_loss, test_acc))

In [ ]:
# # effnet
converter = tf.lite.TFLiteConverter.from_keras_model(effnet_model)
converter.experimental_new_converter = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
effnet_tflite_model = converter.convert()


model_name = "effnet_tflite_model_b3"
open(f"{model_name}.tflite" , "wb").write(effnet_tflite_model)

In [ ]:
#effnet_TL_6c is equal to effnet_tflite_model_b3
effnet_model.save('effnet_TL_6c')